In [30]:
import requests
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
import pandas as pd
import re

In [51]:
# Load the dataset and clean data to drop grants covering 2 places, regional, or statewide
fundsum = pd.read_excel('Invest.gov 10.3.2024.xlsx', sheet_name='FundingSummary')
fundsum['City'] = fundsum['City'].str.strip()
fundsum = fundsum.dropna(subset=['City'])
fundsum = fundsum[~fundsum['City'].str.contains(r'[/\-]')]
fundsum['City'] = fundsum['City'].str.lower()
# Special case mapping for select city names not found in deccenial data
special_mappings = {
    'east boston': 'boston',
    'east weymouth': 'weymouth',
    'north raynham': 'raynham',
    'north truro': 'truro',
}
fundsum['City'] = fundsum['City'].replace(special_mappings)
exclude_names = ['regional', 'statewide', 'metrowest regional transit authority']
fundsum['City'] = fundsum['City'].str.split(',').str[0]
fundsum['City'] = fundsum['City'].str.strip()
fundsum = fundsum[~fundsum['City'].isin(exclude_names)]

In [52]:
# Load data from api and clean sub-county names to match fundsum
url = "https://api.census.gov/data/2020/dec/pl?get=NAME,GEO_ID&for=county%20subdivision:*&in=state:25"
gurl = "geo_id.json"
census_data = pd.read_json(gurl)
census_data.columns = census_data.iloc[0]
census_data = census_data.drop(0)
census_data = census_data[['NAME', 'GEO_ID']]
census_data['NAME'] = census_data['NAME'].str.lower()
census_data['NAME'] = census_data['NAME'].str.split(',').str[0].str.strip()
census_data['NAME'] = census_data['NAME'].str.replace(r'\s*city\s*$', '', regex=True, flags=re.IGNORECASE)
census_data['NAME'] = census_data['NAME'].str.replace(r'\s*town\s*$', '', regex=True, flags=re.IGNORECASE)
census_data['NAME'] = census_data['NAME'].str.strip()
unique_names = census_data['NAME'].unique()

In [53]:
# Merge dataset and display statistics of it
merged_data = pd.merge(fundsum, census_data, left_on='City', right_on='NAME', how='left')
total_rows = len(merged_data)
null_geo_id_count = merged_data['GEO_ID'].isna().sum()
null_geo_id_percentage = (null_geo_id_count / total_rows) * 100
print(f"Total number of rows: {total_rows}")
print(f"Number of rows with null GEO_ID: {null_geo_id_count}")
print(f"Percentage of rows with null GEO_ID: {null_geo_id_percentage:.2f}%")
merged_data.head()

Total number of rows: 444
Number of rows with null GEO_ID: 12
Percentage of rows with null GEO_ID: 2.70%


,Unique ID,Agency Name,Bureau Name,Program Name,Category,Subcategory,Project Name,City,County,State,Funding Amount Excluding Loans,Funding Source,Program Type,Project on Map,Historic ID,NAME,GEO_ID
0,NaN,Department of Transportation,DOT Office of the Secretary,Safe Streets and Roads for All,Transportation,Safety,"Town of Abington, MA Planning and Demonstratio...",abington,Plymouth,Massachusetts,301804.0,BIL,Discretionary,NaN,NaN,abington,0600000US2502300170
1,15591,Department of Homeland Security,Federal Emergency Management Agency,Building Resilient Infrastructure and Communit...,Resilience,Resilience,Acton - Jenks Conservation Land Culvert Improv...,acton,Middlesex,Massachusetts,67500.0,BIL,Formula,Y,P-16587,acton,0600000US2501700380
2,40746,Environmental Protection Agency,Office of Environmental Justice and External C...,Environmental and Climate Justice Block Grants...,Environmental Remediation,Environmental Remediation,Environmental and Climate Justice Block Grants...,amherst,Hampshire,Massachusetts,500000.0,IRA,Discretionary,Y,IRA-4900,amherst,0600000US2501501370
3,14900,Department of Transportation,Federal Motor Carrier Safety Administration,High Priority Activities Program,Transportation,Safety,Commercial Vehicle Safety: University of Massa...,amherst,Hampshire,Massachusetts,938616.0,BIL,Discretionary,Y,P-15878,amherst,0600000US2501501370
4,39012,Department of Agriculture,Rural Business-Cooperative Service,Rural Energy for America Program (REAP),"Clean Energy, Buildings, and Manufacturing",Clean Energy,FY23 9.6KW RM PV SOLAR,amherst,Hampshire,Massachusetts,11189.0,IRA,Discretionary,Y,IRA-3166,amherst,0600000US2501501370


In [54]:
# List all the city names with a null GEO_ID
cities_with_null_geo_id = merged_data[merged_data['GEO_ID'].isna()]['City'].unique()
print("Cities with null GEO_ID:")
for city in cities_with_null_geo_id:
    print(city)

Cities with null GEO_ID:
devens
forestdale
hyannis
martha's vineyard
vineyard haven
whitinsville
woods hole


In [55]:
# Number of rows
merged_data_cleaned = merged_data[merged_data['GEO_ID'].notna()]
print("Rows after dropping rows with null GEO_ID:")
print(merged_data_cleaned.shape)

Rows after dropping rows with null GEO_ID:
(432, 17)


In [56]:
# Function to retrieve race and income data
def get_all_data_by_location(api_key, state_code, geo_id):
    census_base_url = "https://api.census.gov/data/2020/dec/pl"
    income_base_url = "https://api.census.gov/data/2020/acs/acs5"
    demographic_variables = "P1_001N,P1_002N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N,P1_009N"
    income_variables = "B19013_001E"
    all_data = {}
    demographic_params = {
        "get": demographic_variables,
        "ucgid": geo_id,
        "in": f"state:{state_code}",
        "key": api_key
    }
    income_params = {
        "get": income_variables,
        "ucgid": geo_id,
        "in": f"state:{state_code}",
        "key": api_key
    }
    try:
        response = requests.get(census_base_url, params=demographic_params)
        response.raise_for_status()
        demographic_data = response.json()
        if len(demographic_data) >= 2:
            headers = demographic_data[0]
            values = demographic_data[1]
            demographic_result = dict(zip(headers, values))
            all_data.update(demographic_result)
        else:
            print(f"No demographic data found for GEO_ID: {geo_id}")
    except requests.RequestException as e:
        print(f"Failed to fetch demographic data for GEO_ID: {geo_id}, Error: {e}")
    try:
        response = requests.get(income_base_url, params=income_params)
        response.raise_for_status()
        income_data = response.json()
        if len(income_data) >= 2:
            headers = income_data[0]
            values = income_data[1]
            income_result = dict(zip(headers, values))
            all_data.update(income_result)
        else:
            print(f"No income data found for GEO_ID: {geo_id}")
    except requests.RequestException as e:
        print(f"Failed to fetch income data for GEO_ID: {geo_id}, Error: {e}")
    return all_data

In [58]:
# Retrieves data for all unique MA county GEO_IDs, appends to list
api_key = #API KEY
ma_fip = 25
data_dict = {}
data_list = []
missing_data = []
unique_geo_ids = merged_data_cleaned['GEO_ID'].unique()

for geo_id in unique_geo_ids:
  data = get_all_data_by_location(api_key, ma_fip, geo_id)
  if data is not None:
      data_list.append(data)
  else:
      missing_data.append(geo_id)

In [59]:
df = pd.DataFrame(data_list)
df.head()

,P1_001N,P1_002N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N,P1_009N,ucgid,B19013_001E
0,17062,15892,14152,683,39,428,9,581,1170,0600000US2502300170,106001
1,24021,22404,15320,595,27,6032,6,424,1617,0600000US2501700380,137981
2,39263,35627,24770,2382,141,6995,14,1325,3636,0600000US2501501370,56906
3,36569,34361,27167,769,32,5529,6,858,2208,0600000US2500901465,153315
4,439,396,280,8,102,4,0,2,43,0600000US2500701585,98182


In [60]:
# Moves data to Pandas dataframe, does some basic organization and renaming so that data is easy to reference
df = pd.DataFrame(data_list)
df.set_index('ucgid')
df.rename(columns={"P1_001N" : 'Total_Population',
                   "P1_002N" : 'One_Race',
                   "P1_003N" : 'White_Alone',
                   "P1_004N" : 'Black_or_African_American_alone',
                   "P1_005N" : 'American_Indian_and_Alaska_Native_alone',
                   "P1_006N" : 'Asian_alone',
                   "P1_007N" : 'Native_Hawaiian_and_Other_Pacific_Islander_alone',
                   "P1_008N" : 'Some_Other_Race_alone',
                   "P1_009N" : 'Population_of_two_or_more_races',
                   "B19013_001E" : 'Median_household_income',
                   "ucgid" : 'GEO_ID',
                   "county" : 'County_FIPS',
                   "state" : 'State_FIPS'}, inplace = True)
df = df[["GEO_ID", "Total_Population", "One_Race", "White_Alone", "Black_or_African_American_alone",
         "American_Indian_and_Alaska_Native_alone","Asian_alone","Native_Hawaiian_and_Other_Pacific_Islander_alone",
         "Some_Other_Race_alone","Population_of_two_or_more_races","Median_household_income"]]
df

,GEO_ID,Total_Population,One_Race,White_Alone,Black_or_African_American_alone,American_Indian_and_Alaska_Native_alone,Asian_alone,Native_Hawaiian_and_Other_Pacific_Islander_alone,Some_Other_Race_alone,Population_of_two_or_more_races,Median_household_income
0,0600000US2502300170,17062,15892,14152,683,39,428,9,581,1170,106001
1,0600000US2501700380,24021,22404,15320,595,27,6032,6,424,1617,137981
2,0600000US2501501370,39263,35627,24770,2382,141,6995,14,1325,3636,56906
3,0600000US2500901465,36569,34361,27167,769,32,5529,6,858,2208,153315
4,0600000US2500701585,439,396,280,8,102,4,0,2,43,98182
...,...,...,...,...,...,...,...,...,...,...,...
144,0600000US2500379985,7513,6993,6047,262,12,500,0,172,520,91528
145,0600000US2501781035,40876,37726,30264,2232,68,3469,7,1686,3150,92084
146,0600000US2502782000,206518,183097,110158,30485,1019,14688,81,26666,23421,51647
147,0600000US2501582175,1193,1125,1108,0,0,6,0,11,68,81458


In [61]:
new_data = pd.merge(merged_data_cleaned, df, on='GEO_ID', how='left')
new_data = new_data.drop(columns=[col for col in new_data.columns if 'NAME' in col])
new_data = new_data.replace(r'\n', ' ', regex=True)
print(f"Row count after merge: {new_data.shape[0]}")
new_data.head()

Row count after merge: 432


,Unique ID,Agency Name,Bureau Name,Program Name,Category,Subcategory,Project Name,City,County,State,...,Total_Population,One_Race,White_Alone,Black_or_African_American_alone,American_Indian_and_Alaska_Native_alone,Asian_alone,Native_Hawaiian_and_Other_Pacific_Islander_alone,Some_Other_Race_alone,Population_of_two_or_more_races,Median_household_income
0,NaN,Department of Transportation,DOT Office of the Secretary,Safe Streets and Roads for All,Transportation,Safety,"Town of Abington, MA Planning and Demonstratio...",abington,Plymouth,Massachusetts,...,17062,15892,14152,683,39,428,9,581,1170,106001
1,15591,Department of Homeland Security,Federal Emergency Management Agency,Building Resilient Infrastructure and Communit...,Resilience,Resilience,Acton - Jenks Conservation Land Culvert Improv...,acton,Middlesex,Massachusetts,...,24021,22404,15320,595,27,6032,6,424,1617,137981
2,40746,Environmental Protection Agency,Office of Environmental Justice and External C...,Environmental and Climate Justice Block Grants...,Environmental Remediation,Environmental Remediation,Environmental and Climate Justice Block Grants...,amherst,Hampshire,Massachusetts,...,39263,35627,24770,2382,141,6995,14,1325,3636,56906
3,14900,Department of Transportation,Federal Motor Carrier Safety Administration,High Priority Activities Program,Transportation,Safety,Commercial Vehicle Safety: University of Massa...,amherst,Hampshire,Massachusetts,...,39263,35627,24770,2382,141,6995,14,1325,3636,56906
4,39012,Department of Agriculture,Rural Business-Cooperative Service,Rural Energy for America Program (REAP),"Clean Energy, Buildings, and Manufacturing",Clean Energy,FY23 9.6KW RM PV SOLAR,amherst,Hampshire,Massachusetts,...,39263,35627,24770,2382,141,6995,14,1325,3636,56906


In [62]:
# Separates rows into two groups based on percentage of white people
new_data['White_Percentage'] = new_data.apply(lambda row: "≥79%" if int(row['White_Alone'])/int(row['Total_Population']) >= 0.79
    else "<79%",
    axis=1)
print("Rows White Percentage ≥ 79%: " + str(len(new_data[new_data['White_Percentage'] == '≥79%'])))
print("Rows White Percentage < 79%: " + str(len(new_data[new_data['White_Percentage'] == '<79%'])))

Rows White Percentage ≥ 79%: 194
Rows White Percentage < 79%: 238


In [64]:
# Separates rows into four groups based on median household income, based on these groupings:
#[0, 65000], [65000, 90000], [90000, 115000],  [115000, inf]
new_data['Income_Group'] = new_data['Median_household_income'].apply(lambda row: "<65000" if int(row) < 65000
    else "65000-90000" if 90000 > int(row) > 65000
    else "90000-115000" if 115000 > int(row) > 90000
    else "115000+"
    )
for group in new_data['Income_Group'].unique():
    print(f"Rows in group {group}: " + str(len(new_data[new_data['Income_Group'] == group])))

Rows in group 90000-115000: 82
Rows in group 115000+: 37
Rows in group <65000: 147
Rows in group 65000-90000: 166


In [65]:
# Export new_data to CSV
new_data.to_csv('enriched_data.csv', index=False)
print("CSV file 'enriched_data.csv' has been created.")
files.download('enriched_data.csv')

CSV file 'enriched_data.csv' has been created.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>